In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

We want to be able to take a noisy LFP or intracellular Vm trace, band-pass filter it in the theta band (4 - 10 Hz),<br>
and analyze the phase and amplitude of theta at each time point.<br>
<br>
Eventually, we want to take the intracellular Vm of a hippocampal place cell, and assign theta phase to spikes.<br>
We can also do this with the peaks of the theta-filtered intracellular Vm.<br>
If the phase of these spikes or Vm peaks changes with time or spatial position - this is phase precession!

In [ ]:
duration = 10000. # ms
dt = 1  # ms
t = np.arange(0., duration, dt)

freq = 7 # Hz

For a given input signal (e.g. an oscillating CA3 input or local PV inhibitory interneuron), we want to be able to <br>
independently control:<br>
i) the peak firing rate<br>
ii) the "depth" of the oscillation, or (peak - trough) / peak<br>
iii) the phase of the oscillation peak relative to some reference oscillation

In [ ]:
peak_rate = 10.
theta_depth = 0.8
phase_offset = 0

trace = peak_rate * (theta_depth * (np.cos(2. * np.pi * freq * t / 1000. + phase_offset) + 1.) / 2. + (1 - theta_depth))

In [ ]:
plt.figure()
plt.plot(t, trace)
ylim = plt.ylim()
plt.ylim(0., ylim[1])
plt.ylabel('Firing rate (Hz)')
plt.xlabel('Time (ms)')

Here is the syntax for constructing a bandpass filter, using a sliding window of 2 second duration:

In [ ]:
window_dur = 2. # sec
window_len = int(window_dur * 1000 / dt)
theta_filter = scipy.signal.firwin(window_len, [4., 10.], nyq=1000./2./dt, pass_zero=False)

We will attempt to filter the signal in three ways. First we will try the naive way, with no signal padding:

In [ ]:
un_padded_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], trace, padtype=None)

Band-pass filtered traces are returned with zero mean, so we need to subtract the mean from our<br> 
original trace to superimpose them.

In [ ]:
mean_subtracted_trace = trace - np.mean(trace)

plt.figure()
plt.plot(t, mean_subtracted_trace)
plt.plot(t, un_padded_filtered_trace)
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')

This produces a bad edge artifact, as the window slides past the end of the trace.<br>
<br>
Now let's try three alternative methods to "pad" the start and ends of the trace.<br>
The 'odd' method is a 180 degree "rotation" of the trace around start (and end) point. This is the scipy default.<br>
The 'even' method is to "mirror" the trace on both ends (reflect about the y-axis).<br>
The 'gust' method attemps to find edge padding that makes the forward and backwards filters return the same result.

In [ ]:
pad_len = int(window_len)
odd_padded_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], trace, padtype='odd', padlen=pad_len)
even_padded_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], trace, padtype='even', padlen=pad_len)
gust_method_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], trace, method='gust')

In [ ]:
plt.figure()
plt.plot(t, mean_subtracted_trace, '--', label='original', zorder=1)
plt.plot(t, odd_padded_filtered_trace, label='odd filtered', zorder=0)
plt.plot(t, even_padded_filtered_trace, label='even filtered', zorder=0)
plt.plot(t, gust_method_filtered_trace, label='gust filtered', zorder=0)
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

We can also analyze the residual error between the filtered traces and the original trace.

In [ ]:
plt.figure()
plt.plot(t, np.subtract(odd_padded_filtered_trace, mean_subtracted_trace), label='odd filtered')
plt.plot(t, np.subtract(even_padded_filtered_trace, mean_subtracted_trace), label='even filtered')
plt.plot(t, np.subtract(gust_method_filtered_trace, mean_subtracted_trace), label='gust filtered')
plt.ylabel('Residual error')
plt.xlabel('Time (ms)')
plt.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
print('odd filtered:', np.sum(np.abs(np.subtract(odd_padded_filtered_trace, mean_subtracted_trace))))
print('even filtered:', np.sum(np.abs(np.subtract(even_padded_filtered_trace, mean_subtracted_trace))))
print('gust filtered:', np.sum(np.abs(np.subtract(gust_method_filtered_trace, mean_subtracted_trace))))

It appears that mirror-padding ('even' padtype) produces the smallest amplitude edge artifact.

Now we want to take an oscillating band-pass filtered signal as a reference (like an LFP), and assign an<br>
oscillation phase (or angle) to each time point. We can use the Hilbert transformation for this.<br>
The input to the Hilbert transformation should be a mean subtracted signal. Our filter output<br>
should already meet this criterion:

In [ ]:
reference = even_padded_filtered_trace
signal = scipy.signal.hilbert(reference)
envelope = np.abs(signal)
phase = np.angle(signal)

plt.figure()
plt.plot(t, signal, label='signal')
plt.plot(t, envelope, label='envelope')        
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')

plt.figure()
plt.plot(t, phase)
plt.ylabel('Angle (rads)')
plt.xlabel('Time (ms)')

This phase ranges from -pi to +pi. We could also convert it to be between 0 and 360 degrees:

In [ ]:
phase_degrees = 360. / 2. / np.pi * (phase + np.pi)

plt.figure()
plt.plot(t, phase_degrees)
plt.ylabel('Angle (degrees)')
plt.xlabel('Time (ms)')

Now let's consider the above phase vs. time to be our reference (like an LFP).<br>
Now let's consider a recording of intracellular Vm - we want to take the peaks of the oscillation and determine<br>
their phase with respect to the reference (LFP) oscillation.

In [ ]:
def get_osc(t, freq, depth, phase_offset):
    return depth * (np.cos(2. * np.pi * freq * t / 1000. + phase_offset) + 1.) / 2. + (1 - depth)

In [ ]:
theta_Vm = get_osc(t, 7., 1., np.pi / 2)

plt.figure()
plt.plot(t, theta_Vm)
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')

We can use the scipy method find_peaks to find the locations of relative local peaks in our trace.

In [ ]:
peak_indexes = scipy.signal.find_peaks(theta_Vm)

In [ ]:
type(peak_indexes), len(peak_indexes[0])

Each element in this returned tuple corresponds to 1 axis. The first (and only) axis here contained 70 peaks.

In [ ]:
plt.figure()
plt.plot(t, theta_Vm)
plt.plot(t[peak_indexes[0]], theta_Vm[peak_indexes[0]], '.')
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')

Now we can look up the reference theta phase of these intracellular Vm peaks:

In [ ]:
plt.figure()
plt.plot(t[peak_indexes[0]], phase_degrees[peak_indexes[0]], '.')
plt.ylabel('Theta phase (degrees)')
plt.xlabel('Time (ms)')
plt.ylim(0., 360.)

This shows that our method can correctly report the offset phase of a signal that is a constant oscillation<br>
at the same frequency as the reference.<br>
<br>
What about a signal that is oscillating slightly faster than the reference?

In [ ]:
theta_Vm = get_osc(t, 7.2, 1., 0.)
peak_indexes = scipy.signal.find_peaks(theta_Vm)

plt.figure()
plt.plot(t[peak_indexes[0]], phase_degrees[peak_indexes[0]], '.')
plt.ylabel('Theta phase (degrees)')
plt.xlabel('Time (ms)')
plt.ylim(0., 360.)

That looks like phase precession! Except this is for a non-place cell that has a constant mean rate.

What about a place cell with intracellular Vm that has both a slow spatial ramp depolarization and a theta oscillation?

In [ ]:
velocity = 20.  # cm / s
position = t / 1000. * velocity

def get_gaussian_rate(x, peak_loc, width, wrap=True):
    if wrap:
        dx = x[1] - x[0]
        length = dx * len(x)
        extended_x = np.concatenate([x - length, x, x + length])
    else:
        extended_x = x
    sigma = width / 3. / np.sqrt(2.)
    rate = np.exp(-((extended_x - peak_loc) / sigma) ** 2)
    if wrap:
        rate = rate[len(x):-len(x)]
    return rate

In [ ]:
place_field = get_gaussian_rate(position, 100., 60.)

plt.figure()
plt.plot(position, place_field)
plt.ylabel('Amplitude')
plt.xlabel('Position (cm)')

Now we can multiply this spatial rate by a temporal theta modulation factor:

In [ ]:
place_cell_theta = get_osc(t, 7., 0.7, 0.)

place_cell_rate = place_cell_theta * place_field

theta_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], place_cell_rate, padtype='even', padlen=pad_len)

peak_indexes = scipy.signal.find_peaks(theta_filtered_trace, height=0.001)

normalized_reference = 0.5 * reference / np.max(reference) + 0.5

plt.figure()
plt.plot(t, normalized_reference, label='reference')
plt.plot(t, place_cell_rate, label='Vm')
plt.plot(t, theta_filtered_trace, label='theta Vm')
ylim = plt.ylim()
plt.vlines(t[peak_indexes[0]], ymin=ylim[0], ymax=ylim[1], color='grey', linestyle='dashed', alpha=0.5)
plt.ylim(ylim)
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

It appears that every one of the peaks of the theta-filted Vm is aligned to the peaks of the reference.

In [ ]:
plt.figure()
plt.plot(t[peak_indexes[0]], phase_degrees[peak_indexes[0]], '.')
plt.ylabel('Theta phase (degrees)')
plt.xlabel('Time (ms)')
plt.ylim(0., 360.)
plt.xlim(0., duration)

There is very little phase precession, mostly due to a slight warping of the oscillation by the spatial ramp.<br>
<br>
What if the Vm was oscillating slightly faster than the reference?

In [ ]:
place_cell_theta = get_osc(t, 7.2, 0.7, 0.)

place_cell_rate = place_cell_theta * place_field

theta_filtered_trace = scipy.signal.filtfilt(theta_filter, [1.], place_cell_rate, padtype='even', padlen=pad_len)

peak_indexes = scipy.signal.find_peaks(theta_filtered_trace, height=0.001)

normalized_reference = 0.5 * reference / np.max(reference) + 0.5

plt.figure()
plt.plot(t, normalized_reference, label='reference')
plt.plot(t, place_cell_rate, label='Vm')
plt.plot(t, theta_filtered_trace, label='theta Vm')
ylim = plt.ylim()
plt.vlines(t[peak_indexes[0]], ymin=ylim[0], ymax=ylim[1], color='grey', linestyle='dashed', alpha=0.5)
plt.ylim(ylim)
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.legend(frameon=False, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
plt.figure()
plt.plot(t[peak_indexes[0]], phase_degrees[peak_indexes[0]], '.')
plt.ylabel('Theta phase (degrees)')
plt.xlabel('Time (ms)')
plt.ylim(0., 360.)
plt.xlim(0., duration)